In [ ]:
# ================================
# Tuana Bayazıt
# byztuana@gmail.com
# ================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib

df = pd.read_excel("Talent_Academy_Case_DT_2025.xlsx")

print("Before removing duplicates:", df.shape)
df = df.drop_duplicates()
print("After removing duplicates:", df.shape)
df = df.drop(columns=["HastaNo"])

initial_num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
initial_cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

num_imputer = SimpleImputer(strategy='median')
df[initial_num_cols] = num_imputer.fit_transform(df[initial_num_cols])

df[initial_cat_cols] = df[initial_cat_cols].fillna('Unknown')

if 'Cinsiyet' in df.columns and df['Cinsiyet'].nunique() == 2:
    le = LabelEncoder()
    df['Cinsiyet_enc'] = le.fit_transform(df['Cinsiyet'])

multi_cat_cols = ['KanGrubu', 'Uyruk', 'Bolum']
df = pd.get_dummies(df, columns=[col for col in multi_cat_cols if col in df.columns], drop_first=True)


for col in ['KronikHastalik', 'Alerji', 'Tanilar']:
    if col in df.columns:
        # Top 10 frequent items for each column
        top_items = pd.Series([item for sublist in df[col].dropna().apply(lambda x: str(x).split(',')) for item in sublist]).value_counts().head(10).index
        for item in top_items:
            df[f"{col}_{item.strip()}"] = df[col].apply(lambda x: 1 if pd.notnull(x) and item in str(x) else 0)

num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

X = df.drop(columns=["TedaviSuresi"])
y = df["TedaviSuresi"]

cols_to_drop_from_X = ['KronikHastalik', 'Alerji', 'Tanilar']
X = X.drop(columns=[col for col in cols_to_drop_from_X if col in X.columns])

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df.to_csv("preprocessed_case_study.csv", index=False)
print("Preprocessed CSV saved: preprocessed_case_study.csv")

# Optional: Create preprocessing pipeline for future modeling
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

preprocessor.fit(X)
joblib.dump(preprocessor, "preprocessor.joblib")
print("Preprocessor pipeline saved: preprocessor.joblib")

df.head()
df.info()

